In [1]:
import torch
from tqdm.auto import tqdm
from transformer_lens import HookedTransformer
from jaxtyping import Float, Int, Bool
from torch import Tensor
from tqdm.auto import tqdm
import plotly.io as pio
import ipywidgets as widgets
from IPython.display import display, clear_output
import pandas as pd
import numpy as np
import plotly.express as px 
from collections import defaultdict
import matplotlib.pyplot as plt
import re
from IPython.display import display, HTML
from datasets import load_dataset
from collections import Counter
import pickle
import os
import haystack_utils
from transformer_lens import utils
from fancy_einsum import einsum
import einops
import json
import ipywidgets as widgets
from IPython.display import display
from datasets import load_dataset
import random
import math
import random
import neel.utils as nutils
from neel_plotly import *
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
import probing_utils
import pickle
from sklearn.metrics import matthews_corrcoef
import gzip
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotting_utils
import re

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)
random.seed(SEED)

pio.renderers.default = "notebook_connected+notebook"
device = "cuda" if torch.cuda.is_available() else "cpu"
#torch.autograd.set_grad_enabled(False)
#torch.set_grad_enabled(False)

%reload_ext autoreload
%autoreload 2

In [8]:
import requests
import json
import pyzstd

def download_file_with_progress(url, filename):
    response = requests.get(url, stream=True)
    file_size = int(response.headers.get('content-length', 0))
    
    with open(filename, 'wb') as f, tqdm(
        desc=filename,
        total=file_size,
        unit='B',
        unit_scale=True,
        unit_divisor=1024,
    ) as bar:
        for data in response.iter_content(chunk_size=1024):
            bar.update(len(data))
            f.write(data)

# Download the file
url = "https://huggingface.co/datasets/monology/pile-uncopyrighted/resolve/main/train/00.jsonl.zst"
#url = "https://huggingface.co/datasets/monology/pile-uncopyrighted/resolve/main/val.jsonl.zst"
# response = requests.get(url)
# with open("europarl_temp.jsonl.zst", "wb") as f:
#     f.write(response.content)

download_file_with_progress(url, "europarl_temp.jsonl.zst")

# Decompress the file
with open("europarl_temp.jsonl.zst", "rb") as compressed, open("europarl_temp.jsonl", "wb") as decompressed:
    decompressed.write(pyzstd.decompress(compressed.read()))

# Read JSON lines from the decompressed file
with open("europarl_temp.jsonl", "r") as f:
    data = [json.loads(line) for line in f.readlines()]

print(len(data))

europarl_temp.jsonl.zst:   0%|          | 0.00/10.4G [00:00<?, ?B/s]

In [2]:
# Read JSON lines from the decompressed file
with open("europarl_temp.jsonl", "r") as f:
    data = [json.loads(line) for line in f.readlines()]


In [3]:
for example in data:
    if example["meta"]["pile_set_name"] == "EuroParl":
        print(example)
        break

{'text': '2. Označování pneumatik s ohledem na palivovou účinnost (\nIvo Belet\nzpravodaj. - Pane předsedo, v krátkosti bych rád poděkoval švédskému předsednictví za výbornou spolupráci. Domnívám se, že tato smlouva je zároveň ambiciózní i realistická. Přesvědčí nás přiklonit se k větší palivové účinnosti, menší hlučnosti a samozřejmě větší bezpečnosti. Mé poděkování směřuje i ke Komisi a stínovým zpravodajům, Matthiasi Grootemu a Jorgu Chatzimarkakisovi, za jejich skvělou práci.\n', 'meta': {'pile_set_name': 'EuroParl'}}
